# MCP Gateway with LLM Integration

Demonstrates how LLMs use MCP tools to execute code safely.

**What you'll learn:**
- Start HTTP MCP server
- LLM calls tools via MCP
- Workflow orchestration with tool calling

## Step 1: Start HTTP MCP Server

First, start the server in background:

In [ ]:
// Start HTTP server in background
const server = new Deno.Command("deno", {
  args: ["run", "--allow-all", "../examples/http-server.ts"],
  stdout: "piped",
  stderr: "piped"
});

const serverProcess = server.spawn();

// Wait for server to be ready
await new Promise(r => setTimeout(r, 2000));

// Check health
const health = await fetch("http://localhost:3000/health");
const healthData = await health.json();

console.log("✅ MCP Server ready:", healthData);

## Step 2: List Available Tools

See what tools the MCP server exposes:

In [ ]:
const response = await fetch("http://localhost:3000/tools/list", {
  method: "POST"
});

const { tools } = await response.json();

console.log(`📋 Available tools (${tools.length}):`);
for (const tool of tools) {
  console.log(`\n🔧 ${tool.name}`);
  console.log(`   ${tool.description}`);
}

## Step 3: Call Tool Directly

Test tool execution without LLM:

In [ ]:
const response = await fetch("http://localhost:3000/tools/call", {
  method: "POST",
  headers: { "Content-Type": "application/json" },
  body: JSON.stringify({
    tool: "execute_code",
    params: {
      code: "return Array.from({length: 10}, (_, i) => i * i)"
    }
  })
});

const result = await response.json();
console.log("Result:", JSON.stringify(result, null, 2));

## Step 4: LLM with Tool Calling

Now let the LLM decide when to use tools:

In [ ]:
// Import LLM provider
import { createLLM, generateCompletion } from "../examples/llm-provider.ts";
import { generateText, tool } from "npm:ai";
import { z } from "npm:zod";

const apiKey = Deno.env.get("ANTHROPIC_API_KEY") ||
               Deno.env.get("OPENAI_API_KEY") ||
               Deno.env.get("GOOGLE_API_KEY");

if (!apiKey) {
  throw new Error("Set an API key in llm-demo.ipynb first!");
}

const model = createLLM({ apiKey });

console.log("✅ LLM ready");

Define the tool for the LLM:

In [ ]:
const executeCodeTool = tool({
  description: "Execute TypeScript/JavaScript code safely in a sandbox",
  parameters: z.object({
    code: z.string().describe("TypeScript/JavaScript code to execute"),
    context: z.any().optional().describe("Optional context data")
  }),
  execute: async ({ code, context }) => {
    const response = await fetch("http://localhost:3000/tools/call", {
      method: "POST",
      headers: { "Content-Type": "application/json" },
      body: JSON.stringify({
        tool: "execute_code",
        params: { code, context }
      })
    });
    const result = await response.json();
    return result.result;
  }
});

## Step 5: Ask LLM to Solve Task with Code

The LLM will generate and execute code:

In [ ]:
const result = await generateText({
  model,
  tools: { execute_code: executeCodeTool },
  maxSteps: 5,
  prompt: "Calculate the sum of all prime numbers between 1 and 100. Write and execute TypeScript code to do this."
});

console.log("\n🤖 LLM Response:");
console.log(result.text);

console.log("\n📊 Tool Calls:");
for (const step of result.steps) {
  if (step.toolCalls) {
    for (const call of step.toolCalls) {
      console.log(`\n🔧 Called: ${call.toolName}`);
      console.log("   Args:", JSON.stringify(call.args, null, 2));
      console.log("   Result:", JSON.stringify(step.toolResults, null, 2));
    }
  }
}

## Step 6: Multi-Step Workflow

LLM orchestrates multiple tool calls:

In [ ]:
const workflow = await generateText({
  model,
  tools: { execute_code: executeCodeTool },
  maxSteps: 10,
  prompt: `Analyze this sales data step by step:
1. Calculate total revenue
2. Find best selling product
3. Calculate average price

Data: [{"product":"Laptop","price":1200,"qty":2},{"product":"Mouse","price":25,"qty":5},{"product":"Keyboard","price":80,"qty":3}]

Write code to perform each calculation.`
});

console.log("\n🤖 Workflow Result:");
console.log(workflow.text);

console.log("\n📈 Steps executed:", workflow.steps.length);

## Step 7: Cleanup

Stop the MCP server:

In [ ]:
serverProcess.kill("SIGTERM");
await serverProcess.status;
console.log("✅ Server stopped");

## Summary

**What we demonstrated:**
- ✅ MCP server exposes tools via HTTP
- ✅ LLM discovers and calls tools automatically
- ✅ Safe code execution in sandbox
- ✅ Multi-step workflow orchestration
- ✅ LLM decides when and how to use tools

**The MCP Pattern:**
```
User Query → LLM → Tool Selection → MCP Server → Sandbox → Results → LLM → User
```

This is how Claude Code, Cline, and other AI coding tools work!